# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [25]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostClassifier, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [13]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [14]:
spaceship.dropna(inplace=True)
spaceship[['Deck', 'CabinNum', 'Side']] = spaceship['Cabin'].str.split('/', expand=True)
spaceship.drop(columns=['PassengerId', 'Name', 'Cabin'], inplace=True)
spaceship['CabinNum'] = spaceship['CabinNum'].astype(int)
spaceship = pd.get_dummies(spaceship, drop_first=True)
spaceship['Transported'] = spaceship['Transported'].astype(int)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [26]:
features = spaceship.drop(columns='Transported')
target = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

forest = RandomForestClassifier(n_estimators=100, max_depth=8)

forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=8)

- Evaluate your model

In [16]:
pred_test = forest.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_test))
print("\nClassification Report:\n", classification_report(y_test, pred_test))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_test))

Accuracy: 0.791981845688351

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.78      0.79       661
           1       0.79      0.80      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322


Confusion Matrix:
 [[516 145]
 [130 531]]


In [17]:
pred_train = forest.predict(X_train)

print("Accuracy:", accuracy_score(y_train, pred_train))
print("\nClassification Report:\n", classification_report(y_train, pred_train))
print("\nConfusion Matrix:\n", confusion_matrix(y_train, pred_train))

Accuracy: 0.8433005299015897

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.83      0.84      2618
           1       0.84      0.86      0.85      2666

    accuracy                           0.84      5284
   macro avg       0.84      0.84      0.84      5284
weighted avg       0.84      0.84      0.84      5284


Confusion Matrix:
 [[2169  449]
 [ 379 2287]]


In [18]:
model = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(model, X_train, y_train, cv=5)

In [19]:
print("Accuracy for each fold:", scores)
print("Average accuracy:", scores.mean())

Accuracy for each fold: [0.82119205 0.81362346 0.79848628 0.79943236 0.79734848]
Average accuracy: 0.8060165276224879


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [20]:
grid = {"n_estimators": [50, 100, 500,  1000],
        "estimator__max_leaf_nodes": [250, 500, 250],
        "estimator__max_depth":[5,8,12]}

- Run Grid Search

In [24]:
ada_class = AdaBoostClassifier(DecisionTreeClassifier())
model = GridSearchCV(estimator = ada_class, param_grid = grid, cv=5, verbose=5)

model.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV 1/5] END estimator__max_depth=5, estimator__max_leaf_nodes=250, n_estimators=50;, score=0.754 total time=   0.7s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV 2/5] END estimator__max_depth=5, estimator__max_leaf_nodes=250, n_estimators=50;, score=0.759 total time=   0.6s


...

[CV 4/5] END estimator__max_depth=12, estimator__max_leaf_nodes=250, n_estimators=1000;, score=0.766 total time=  15.1s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV 5/5] END estimator__max_depth=12, estimator__max_leaf_nodes=250, n_estimators=1000;, score=0.773 total time=  15.3s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier()),
             param_grid={'estimator__max_depth': [5, 8, 12],
                         'estimator__max_leaf_nodes': [250, 500, 250],
                         'n_estimators': [50, 100, 500, 1000]},
             verbose=5)

- Evaluate your model

In [27]:
model.best_params_
best_model = model.best_estimator_

In [28]:
pred_test = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_test))
print("\nClassification Report:\n", classification_report(y_test, pred_test))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_test))

Accuracy: 0.7760968229954615

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.81      0.78       661
           1       0.79      0.75      0.77       661

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322


Confusion Matrix:
 [[533 128]
 [168 493]]


In [29]:
pred_train = best_model.predict(X_train)

print("Accuracy:", accuracy_score(y_train, pred_train))
print("\nClassification Report:\n", classification_report(y_train, pred_train))
print("\nConfusion Matrix:\n", confusion_matrix(y_train, pred_train))

Accuracy: 0.9998107494322483

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2618
           1       1.00      1.00      1.00      2666

    accuracy                           1.00      5284
   macro avg       1.00      1.00      1.00      5284
weighted avg       1.00      1.00      1.00      5284


Confusion Matrix:
 [[2617    1]
 [   0 2666]]
